In [8]:
#This program creates a simluation of the clustering algorithm based on the Potts model from statistical physics. 
#The algorithm is based on the paper "Blatt, M., Wiseman, S. and Domany, E. Clustering data through an analogy to the Potts model.
#Advances in Neural Information Processing Systems, pp.416-422, 1996."

In [9]:
#In this block, we assign the initial position and spin configurations of 10 particles. 
#We create two lists (arrays) each of size 10; the first has the position coordinates of 10 particles. The second has spin data 
# for the particles. 

import random
import numpy as np
q=2
N=10

Position_2=[[] * 2 for i in range(10)]
#print Position_2
Spin=[]
for counter in range(0,10):
    a=random.randint(1,10)
    b=random.randint(1,10)
    Position_2[counter]=([a,b])  
    Spin.append(random.randint(1,3))          #Random value between 1 to 5 is assigned to spin array.
#print Position_2, Spin


In [10]:
#Create the Individual neighbourhood value matrix. In this cell, we compute the immediate two neighbours of each element in the Position list.
Position_2=[[6, 4], [3, 1], [8, 3], [3, 9], [4, 4], [4, 7], [2, 10], [5, 5], [7, 4], [10, 6]]
import numpy
Neighbours=numpy.zeros(shape=(10,10))                       #Initializing Neighbours matrix to 0s
Dist_Matrix=numpy.zeros(shape=(10,10))
for i in range(0,10):                                       #Counter for row.
    nbr_1=i                                                 #Just setting initial neighbours to dummmy values. It's imprtant to 
                                                            #set as i otherwise, it messes up the loops ahead 
    nbr_2=i
    dist_1=1000000                                             #Initial large dummy distances. Important to not keep them as 0.                  
    dist_2=1000000
   
    for j in range(0,10):
        temp_dist=abs((Position_2[i][0]-Position_2[j][0])**2+(Position_2[i][1]-Position_2[j][1])**2)
        #print "temp_dis", temp_dist
        Dist_Matrix[i][j]=temp_dist
        if temp_dist<=dist_1 and temp_dist>0:               #If the new distance is smaller than both the existing distances
            Neighbours[i][nbr_2]=0
            Neighbours[i][j]=1
            nbr_2=nbr_1
            nbr_1=j
            dist_2=dist_1
            dist_1=temp_dist
        if temp_dist>dist_1 and temp_dist<=dist_2:          #If the new distance is in between the two existing distances
            Neighbours[i][nbr_2]=0
            Neighbours[i][j]=1
            nbr_2=j
            dist_2=temp_dist
#print "Distance Matrix", Dist_Matrix
#print "Individual enighbours", Neighbours            
            
            
            
        

In [11]:
#In this block we determine the mutual neighbourhoods of each element.

Mut_Nbr=numpy.zeros(shape=(10,10))
print Mut_Nbr[0][1]
for i in range(0,10):
    for j in range(0,10):
        #print Neighbours[0][1]*Neighbours[1][0]
        if Neighbours[i][j]*Neighbours[j][i]==1:
            Mut_Nbr[i][j]=1
            Mut_Nbr[j][i]=1
print "Mutual neighbours"
print Mut_Nbr            

0.0
Mutual neighbours
[[ 0.  0.  0.  0.  0.  0.  0.  1.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  1.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 1.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


In [12]:
#In this cell, we compute the interaction strengths between mutual neighbours


#First, we compute \hat{K} (call this KK in the program), the "average number of neighbours per site". 
#We compute this by the formula given in section 2

import math
tot_inter=0
for i in range(0,10):
    for j in range(0,10):
        if Mut_Nbr[i][j]==1:
            tot_inter+=1
KK= 2*tot_inter/10
#print KK

#print Dist_Matrix
#Next, we use the formula for the mutual interactions to compute the mutual interactions matrix J
#J=[[0.0 for i in xrange(10)] for j in xrange(10)]
J=numpy.zeros(shape=(10,10))
#print J
for i in range(0,10):
    for j in range(0,10):
        if Mut_Nbr[i][j]==1:
            dd=Dist_Matrix[i][j]
            #print "dd", dd
            J[i][j]=round(1/(math.exp(dd)*KK),8)
            J[j][i]=J[i][j]
            #print "check i,j",i,j, math.exp(J[i][j]/0.001)
    #print J[i]   

print "Mutual Interaction Matrix"
print J

Mutual Interaction Matrix
[[ 0.          0.          0.          0.          0.          0.          0.
   0.06766764  0.18393972  0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.06766764  0.        ]
 [ 0.          0.          0.          0.          0.          0.00336897
   0.06766764  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.06766764  0.          0.        ]
 [ 0.          0.          0.          0.00336897  0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.06766764  0.          0.          0.
   0.          0.          0.        ]
 [ 0.06766764  0.          0.          0.          0.06766764  0.          0.
   0.          0.          0.        ]
 [ 0.18393972  0.          0.0

In [ ]:
#In this cell, we implement the Monte Carlo method on our sample set of points.

import matplotlib.pyplot as plt

#We create a bond matrix that has states 1 and 0 at the i,j th position if there is a bond between x_i and x_j
Bond_Matrix=numpy.empty(shape=(10,10))
switer=15   #We will run the Monte Carlo simulation 15 times


T=numpy.arange(0.001,100000.001, 10000)   #Temperature range

vari=numpy.empty(shape=len(T))
chi=numpy.empty(shape=len(T))
for temp_count in range(0,len(T)): 
    m=[[] for hh in range(switer+1)]
    m_Sq=[[] for yy in range(switer+1)]
    tot_m=0
    tot_m_Sq=0
    for SW_counter in range(0,switer+1):
        J=[[0,0,0,0,0,0,0,0.067,0.184,0],
       [0,0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0.067,0],
       [0,0,0,0,0,0.0033,0.067,0,0,0],
       [0,0,0,0,0,0,0,0.067,0,0],
       [0,0,0,0.0034,0,0,0,0,0,0],
       [0,0,0,0.067,0,0,0,0,0,0],
       [0.067,0,0,0,0.067,0,0,0,0,0],
       [0.1839,0,0.067,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0,0]]
        #print "Temperature=", T[temp_count]
        for i in range(0,9):
            for j in range(i+1,10):
                #Calculate the probability of deleting a bond
                if Spin[i]==Spin[j] and J[i][j]!=0:
                    #rint "check pd", math.exp(J[i][j]/0.1)
                    Pd=0.000005/math.exp(J[i][j]/T[temp_count])
                    #Pd=0.5
                else:
                    Pd=1
                dummy=random.uniform(0,1)
                #print "i=", i
                #print "j=", j
                #print "deletion prob", Pd
                #print "dummy", dummy
                if dummy>Pd:
                    Bond_Matrix[i][j]=1
                    Bond_Matrix[j][i]=1
                else:
                    Bond_Matrix[i][j]=0
        #print "Bond Matrix"
        #print Bond_Matrix            
        #Now we make clusters based on whether there is a bond between given particles or not
        Cluster_Matrix=J=[[-1 for c1 in xrange(10)] for c2 in xrange(10)]
        i=0 
        clus_num=-1
        cluster=[]
        cluster_reserve=[]
        l=0
        clus_size=[[] for h in range(10)]
        while (i<=9):
            #print "i=",i
            #cluster=[]
            cluster.append(i)
            clus_num +=1
            Cluster_Matrix[clus_num][0]=i
            clus_size[clus_num]=1
            k=0
            j=-1
            while j<k:   #scouting for the k-th level of friends
                for count in range(i+1,10):
                    #print "Bond_Matrix element", Bond_Matrix[cluster[j+1]][count]
                    if Bond_Matrix[cluster[j+1]][count]==1:
                        if count in cluster:
                            #print count, "is already there"
                            g=1
                        else:
                            #print "main element",i, "j+1=", j+1, "the element we are adding to the cluster", count, "k=", k
                            #print "[i]cluster[j+1]=",i, cluster[j+1]#, "bond matrix", Bond_Matrix[cluster[j+1]][count]
                            cluster.append(count)
                            #print "clus_num=", clus_num, "k=", k
                            Cluster_Matrix[clus_num][k+1]=count
                            #print "Added to Cluster Matrix", Cluster_Matrix[clus_num][k+1]
                            k+=1 
                            clus_size[clus_num]+=1
                j+=1 
            #print "Cluster", clus_num, Cluster_Matrix[clus_num]
            i+=1
            #print "Cluster Matrix", Cluster_Matrix
            #print "CM", Cluster_Matrix[0][1]

            if i in cluster:
                i+=1

        #print "Cluster Matrix", Cluster_Matrix

        del clus_size[clus_num+1:]

        #print "Cluster sizes", clus_size
        for a in range(0, clus_num):
            b=0
            rr=random.randint(1,2)
            while Cluster_Matrix[a][b]!=-1:
                Spin[Cluster_Matrix[a][b]]=rr
                #print "aa"
                b+=1
        #print "Spin", Spin        
        N_max=max(clus_size)
        #print "N_max", N_max
        # Now we calculate the magentization according to the formula given in Page 420, section 4
        N=10
        q=2
        m[SW_counter]=(q*N_max/10 - 1)/(q-1)
        m_Sq[SW_counter]=m[SW_counter]**2
        #print "m", m, "m_SQ", m_Sq
        tot_m=tot_m+m[SW_counter]
        tot_m_Sq=tot_m_Sq+m_Sq[SW_counter]
    print "Clusters at temperature ",T[temp_count], "are:"
    print Cluster_Matrix
    m_avg_sq=(tot_m/switer)^2
    m_sq_avg=tot_m_Sq/switer
    #print "mean_sq", mean_sq_avg, "mean_avg_sq",mean_avg_sq
    vari[temp_count]=(N*(m_sq_avg-m_avg_sq )/T[temp_count])
    chi[temp_count]=vari[temp_count]*temp_count/N
    print "variance of magnetisation at temperature",T[temp_count], "is",vari[temp_count]
#print chi
plt.plot(T[1:], vari[1:])
plt.show()
    
    
    
    
    

Clusters at temperature  0.001 are:
[[0, 7, 8, 4, 2, -1, -1, -1, -1, -1], [1, -1, -1, -1, -1, -1, -1, -1, -1, -1], [3, -1, -1, -1, -1, -1, -1, -1, -1, -1], [5, -1, -1, -1, -1, -1, -1, -1, -1, -1], [6, -1, -1, -1, -1, -1, -1, -1, -1, -1], [8, -1, -1, -1, -1, -1, -1, -1, -1, -1], [9, -1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]]
variance of magnetisation at temperature 0.001 is 30000.0
Clusters at temperature  10000.001 are:
[[0, 7, 8, 4, 2, -1, -1, -1, -1, -1], [1, -1, -1, -1, -1, -1, -1, -1, -1, -1], [3, -1, -1, -1, -1, -1, -1, -1, -1, -1], [5, -1, -1, -1, -1, -1, -1, -1, -1, -1], [6, -1, -1, -1, -1, -1, -1, -1, -1, -1], [8, -1, -1, -1, -1, -1, -1, -1, -1, -1], [9, -1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]]
variance of magnetisation at temperature 10000